# 提示语模板

## Mustache

In [ ]:
%pip install chevron

In [1]:
from chevron.renderer import render as mustache_render
from chevron.tokenizer import tokenize as mustache_tokenize

mustache_render(template='Hello, {{ mustache }}! {{ name }}')

'Hello, ! '

## 通过 binding 提前定义 Template

In [2]:
from illufly.types import Template, Messages, Runnable

t = Template("IDEA", binding_map={"task": "hi"})
t.format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\nhi\n'

## 通过 format 提供变量

In [3]:
t.format({"task": "帮我写一首儿歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n帮我写一首儿歌\n'

## 通过绑定 Runnable 提供变量

In [2]:
from illufly.types import Template, Messages, Runnable

class A(Runnable):
    def call(self, prompt: str, **kwargs):
        return prompt

a = A()
t = Template(text="你的任务是：{{last_output}}")
a.bind_consumer(t)

a("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'

In [3]:
a.provider_dict

{'last_output': '帮我写一首儿歌'}

## 绑定中使用函数

In [5]:
# 映射到深层嵌套的字典变量
Template(
    "IDEA",
    binding_map={"task": lambda x: f"请帮我写一首歌"}
).format()

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

## 使用 bind_provider

In [8]:
class A(Runnable):
    def call(self, prompt: str, **kwargs):
        return prompt

a = A()
t = Template(text="你的任务是：{{task}}")
t.bind_provider(a, {"task": "last_output"})

a("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'

## 使用 lazy_binding_map

使用 lazy_binding_map 可以先声明对象，然后在合适的时机自动实现绑定。

In [3]:
from illufly.types import Template, Messages, Runnable

class A(Runnable):
    def call(self, prompt: str, **kwargs):
        return prompt

t = Template(text="你的任务是：{{task}}", lazy_binding_map={"task": "last_output"})

a = A()
a.bind_consumer(t)

a("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'

**对话模型可以使用 lazy_binding_map 实现灵活的多模板绑定：**

In [19]:
from illufly.chat import ChatQwen
from illufly.types import Template

qwen = ChatQwen(
    memory=[
        Template(text="你的任务是：{{mytask}}", lazy_binding_map={"mytask": "task"}),
        Template(text="请认真帮我完成：{{recall}}", lazy_binding_map={"recall": "task"})
    ]
)
qwen.get_chat_memory("写一首歌")

[{'role': 'system', 'content': '你的任务是：写一首歌'},
 {'role': 'user', 'content': '请认真帮我完成：写一首歌'}]